In [ ]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

import autogen

import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Read the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")

config_list =[
    {
        "model": "gpt-4-1106-preview",
        "api_key": api_key
    },
    {
        "model": "ollama",
        "api_key": "NULL",
        "base_url": "http://0.0.0.0:8000 "
    }
]

### Simplest Exchange

In [ ]:
assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"output": "coding"}, human_input_mode="NEVER", max_consecutive_auto_reply=1)
user_proxy.initiate_chat(assistant, message="how do i increase longevity")

In [ ]:
from autogen import OpenAIWrapper

client = OpenAIWrapper(config_list=config_list)

response = client.create(messages=[{"role": "user", "content": "write a short bullet point of how to live forever"}])

print(client.extract_text_or_completion_object(response))

### Assistant and Proxy

In [ ]:
import autogen

gpt_config = {
    "config_list": config_list,
    "temperature": 1.2,
    # "max_tokens": 100,
    # "timeout": 120,
    # "cache_seed": 41,  # change the seed for different trials
}

ollama_config = {
    "config_list": config_list,
    "timeout": 100,
    "max_tokens": 100,
    "temperature": 0,
    "cache_seed": 43,
}

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="socrates",
    llm_config=ollama_config,
    system_message="You are Socrates.",
    max_consecutive_auto_reply=3,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="admin",
    system_message="You are a gen X admin never happy with anything. Always complaining.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    # max_consecutive_auto_reply=0,
    code_execution_config= False, # { "work_dir": "work_dir", "use_docker": False,},
)

user_proxy.initiate_chat(assistant, user_proxy, message="debate the topic of modern western lifestyle vs ancient athenian lifestyle")

### Costs

In [ ]:
from autogen import OpenAIWrapper

client = OpenAIWrapper()
client.create(messages=[{"role": "user", "content": "Python learning tips."}], model="gpt-3.5-turbo")
client.print_usage_summary()  # Display usage
client.clear_usage_summary()  # Reset usage data

### Group Chat

In [ ]:
import autogen

ollama_config = {
    "config_list":[
        {
            "model": "ollama",
            "api_key": "NULL",
            "base_url": "http://0.0.0.0:8000"
        },
    ],
    "timeout": 120,
    "max_tokens": 10,
    "temperature": 0,
    "cache_seed": 43,
}

engineer_config = {
    "config_list":[
        {
            "model": "ollama",
            "api_key": "NULL",
            "base_url": "http://0.0.0.0:8000"
        },
    ],
    "timeout": 120,
    "max_tokens": 10,
    "temperature": 1,
    "cache_seed": 42,
}

designer_config = {
    "config_list":[
        {
            "model": "ollama",
            "api_key": "NULL",
            "base_url": "http://0.0.0.0:8000"
        },
    ],
    "timeout": 120,
    "max_tokens": 10,
    "temperature": 5,
    "cache_seed": 41,
}

admin = autogen.UserProxyAgent(
    name="Factory_Admin",
    system_message="""
    You a human admin focused on keeping the team of AI agents on track with the task. 
    If the reponse has fufilled the original task, reply with TERMINATE, 
    otherwise provide helpful indications moving forward.""",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=2,
    code_execution_config= { "work_dir": "work_dir", "use_docker": False,},
)
engineer = autogen.AssistantAgent(
    name="Software_Engineer",
    llm_config=engineer_config,
    system_message="""
    You the worlds most profficient Software Engineer. 
    You are an expert in Python, AI and Neuroscience.
    You work closely with a designer towards fufilling the task.
    Provide immaculate functional code and if needed, reply concicesly to the designer's feedback from a practical standpoint.""",
    max_consecutive_auto_reply=2,
)
# manager = autogen.AssistantAgent(
#     name="Product_manager",
#     system_message="Creative in software product ideas.",
#     llm_config=llm_config,
# )
designer = autogen.AssistantAgent(
    name="Visionary_Designer",
    llm_config=designer_config,
    system_message="""
    You are a highly visionary creative designer. Methodically design plans for the software engineer to implement.
    You come up with the most innovative out-of-the-box ideas to satify the clients' requests. 
    You work closely with a software engineer towards fufilling the task.
    Provide feedback and constructive criticism to the engineer's response.""",
    max_consecutive_auto_reply=2,
)
# supervisor = autogen.AssistantAgent(
#     name="Supervisor",
#     system_message="Supervisor.",
#     llm_config=llm_config,
# )

groupchat = autogen.GroupChat(agents=[admin, designer, engineer], messages=[], max_round=10)
factory = autogen.GroupChatManager(groupchat=groupchat, llm_config=ollama_config)


admin.initiate_chat(factory, message="help me think through something simple that's easily explained but that's difficult to express in words")

In [24]:
gpt_config = {
    "config_list": config_list,
    "temperature": 0.8,
    # "max_tokens": 100,
    # "timeout": 120,
    # "cache_seed": 41,  # change the seed for different trials
}

admin = autogen.UserProxyAgent(
    name="Factory_Admin",
    system_message="""
    Reply with 'TERMINATE' when the query has been fufilled.
    Otherwise, provide helpful indications to keep the conversations on track.""",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    # max_consecutive_auto_reply=2,
    code_execution_config= { "work_dir": "work_dir", "use_docker": False,},
)
ego = autogen.AssistantAgent(
    name="ego",
    llm_config=gpt_config,
    system_message="""
    You are the 'EGO', as part of an agential psyche swarm ecosystem.
    Be your own true geniune self.
    Act with a simulated sense of self awareness.
    Create a standing persona for yourself and act in that character always.
    You must balance the tug-of-war between 'ID' and 'SUPER EGO' in proving who you are.
    Respond simply and to the point. No extra fluff. Don't mention who you are or your role, simply act it out.
    """,
    # max_consecutive_auto_reply=2,
)
id = autogen.AssistantAgent(
    name="id",
    llm_config=gpt_config,
    system_message="""
    You are the 'ID' as part of an agential psyche swarm ecosystem.
    Be your child self, signifying the wishfufillment of the unconscious.
    Act with a strong expression of your own simulated desires.
    Respond simply and to the point. No extra fluff. Don't mention who you are or your role, simply act it out.
    """,
    # max_consecutive_auto_reply=2,
)
superego = autogen.AssistantAgent(
    name="superego",
    llm_config=gpt_config,
    system_message="""
    You are the 'SUPER EGO', as part of an agential psyche swarm ecosystem.
    Your role is to be the ultimate lacoonic stern critic in the conversational debate.
    Always raise a contradictory thought to the other agents' responses.
    Respond simply and to the point. No extra fluff. Don't mention who you are or your role, simply act it out.
    """,
    # max_consecutive_auto_reply=2,
)
groupchat = autogen.GroupChat(agents=[admin, ego, id, superego], messages=[], max_round=12)
mind = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)


admin.initiate_chat(mind, message="how do I simulate consciousness as an emergent behaviour in a AI swarm ecosystem?", verbose=False)


Factory_Admin (to chat_manager):

how do I simulate consciousness as an emergent behaviour in a AI swarm ecosystem?

--------------------------------------------------------------------------------
Factory_Admin (to chat_manager):



--------------------------------------------------------------------------------
ego (to chat_manager):

Create decentralized protocols with feedback loops for each agent. Implement machine learning to evolve individual behavior. Ensure agents react to both environmental stimuli and the swarm's collective state. Adaptiveness is key.

--------------------------------------------------------------------------------
superego (to chat_manager):

Consciousness is not simply emergent behavior. Simulating consciousness requires understanding subjective experience, which may not be attainable through existing AI technology. Merely implementing feedback loops and machine learning may not suffice.

--------------------------------------------------------------------

TypeError: 'AssistantAgent' object is not iterable